In [1]:
import urllib.request
import requests
from selectorlib import Extractor
import json 
import random
import re
from time import sleep
import os
import jsonlines
import pandas as pd
import datetime

#### Step 1: Read Keyword List 

The keyword list should be in .txt format with each line containing a keyword.

In [2]:
!ls ./../DATASET/KeywordLists

appario_generic.txt                top_100_keyword_list.txt
generic_keywords_cloudtail_top.txt


In [11]:
keywords = open('./../DATASET/KeywordLists/top_100_keyword_list.txt', 'r')
keyword_list = []

for k in keywords:
    k = k.strip("\n")
    keyword_list.append(k)
print('Keyword List: ', keyword_list[:10])
print('Keyword Count: ', len(keyword_list))

Keyword List:  ['nintendo+switch', 'laptop', 'ssd', 'kindle', 'ps4', 'airpods', 'ipad', 'tablet', 'iphone', 'alexa']
Keyword Count:  100


In [4]:
headers = [
           {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
           },
           {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
           },
           {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
           },
           {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
           },
           {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
           },
           {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Safari/605.1.15',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
           },
           {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
           },
           {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
           },
           {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
           },
           {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:70.0) Gecko/20100101 Firefox/70.0',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
           },
           {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:70.0) Gecko/20100101 Firefox/70.0',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
           },
           {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
           },
           {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
           },
           {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36 OPR/68.0.3618.165',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
           },
           {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Trident/7.0; rv:11.0) like Gecko',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
           },
           {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36 Edg/83.0.478.37',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
           }
]

In [5]:
e = Extractor.from_yaml_file('./Extractor/Flipkart_keyword_search_product_list.yml')
l = Extractor.from_yaml_file('./Extractor/Flipkart_nextpg.yml')
p = Extractor.from_yaml_file('./Extractor/Flipkart_keyword_search_product_page.yml')

In [6]:
MAX_TRIALS_A = 50  # Set the max number of trials to perform here.
ERROR_COUNT_A = 1
def scrape_SearchResult(url):  
    global ERROR_COUNT_A
    
    '''
    This function downloads the webpage at the given url using requests module.
    
    Parameters:
    url (string): URL of webpage to scrape
    Returns: 
    string: If the URL contains products, returns the html of the webpage as text, else returns 'False'.
    '''
    
    # Download the page using requests
    print("Downloading %s"%url)
    trial = 0
    while(True):
        # Ask to change vpn every 3 pages without results to ensure data is not missed because of being blocked
        if ERROR_COUNT_A % 3 == 0:
            _ = input('Please Change VPN and enter \'DONE\' to continue')
            ERROR_COUNT_A += 1
        if trial == MAX_TRIALS_A:  
            print("Max trials exceeded yet no Data found on this page!")
            ERROR_COUNT_A += 1
            return 'False'
    
        # Get the html data from the url
        while True:
            try:
                trial = trial + 1
                print("Trial no:", trial)
                r = requests.get(url, headers=random.choice(headers), timeout = 15) 
                
                # We use product_list.yml extractor to extract the product details from the html data text
                data = e.extract(r.text) 
#                 print(data)
                
                if data['url_a'] is None:
                    if data['url_b'] is None:
                        print("Retrying with new user agent!")
                        break
                    else:
                        return r.text
                else:
                    return r.text
                    
            except requests.exceptions.RequestException as err:
                print('Error Detected: ', err)
                print('Retrying after 30 seconds')
                sleep(30)
                
                continue
            except requests.exceptions.HTTPError as err:
                print('Error Detected: ', err)
                print('Retrying after 30 seconds')
                sleep(30)
                
                continue
            except requests.exceptions.ConnectionError as err:
                print('Error Detected: ', err)
                print('Retrying after 30 seconds')
                sleep(30)
                
                continue
            except requests.exceptions.Timeout as err:
                print('Error Detected: ', err)
                print('Retrying after 30 seconds')
                sleep(30)
                
                continue

In [7]:
MAX_TRIALS_B = 50  # Set the max number of trials to perform here.
ERROR_COUNT_B = 1
def scrape_ProductPage(url):  
    '''
    This function downloads the webpage at the given url using requests module.
    
    Parameters:
    url (string): URL of webpage to scrape
    Returns: 
    string: If the URL contains products, returns the html of the webpage as text, else returns 'False'.
    '''
    global ERROR_COUNT_B
    
    # Download the page using requests
    print("Downloading %s"%url)
    trial = 0
    while(True):
        # Ask to change vpn every 20 pages without results to ensure data is not missed because of being blocked
        if ERROR_COUNT_B % 20 == 0:
            _ = input('Please Change VPN and enter \'DONE\' to continue')
            ERROR_COUNT_B += 1
        if trial == MAX_TRIALS_B:  
            print("Max trials exceeded yet no Data found on this page!")
            ERROR_COUNT_B += 1
            return 'False'
        trial = trial + 1
        print("Trial no:", trial)
        
        # Get the html data from the url
        while True:
            try:
                r = requests.get(url, headers=random.choice(headers), timeout = 15) 
                
                # We use product_list.yml extractor to extract the product details from the html data text
                data = p.extract(r.text) 
                # If the products title in the scraped html is not empty, return extracted details as dict. 
                # If the products title in the scraped html is empty, retry with new user agent.
                if data['Title'] != None:
                    return (p.extract(r.text))
                else:
                    print("Retrying with new user agent!")
                    break
            except requests.exceptions.RequestException as err:
                print('Error Detected: ', err)
                print('Retrying after 30 seconds')
                sleep(30)
                continue
            except requests.exceptions.HTTPError as err:
                print('Error Detected: ', err)
                print('Retrying after 30 seconds')
                sleep(30)
                continue
            except requests.exceptions.ConnectionError as err:
                print('Error Detected: ', err)
                print('Retrying after 30 seconds')
                sleep(30)
                continue
            except requests.exceptions.Timeout as err:
                print('Error Detected: ', err)
                print('Retrying after 30 seconds')
                sleep(30)
                continue

In [8]:
FileName = 'SCRAPED_KEYWORD_LIST_TOP_100_FLIPKART_BigSavingsDay_6thTo10thAUG'
# FileName = 'SCRAPED_KEYWORD_LIST_APPARIO_GENERIC'
#FileName = 'SCRAPED_KEYWORD_LIST_APPARIO'
#FileName = 'SCRAPED_KEYWORD_LIST_CLOUDTAIL'

outfile_path = str('./ScriptOutput/DATASET/' + str(FileName) + '.jsonl')  

In [9]:
# Keyword Type
# Example: APPARIO GENERIC
# Example: TOP 100 UK
# Example: TOP 100 INDIA
KeywordType = input("Enter Keyword Type")

Enter Keyword TypeTop 100 Flipkart BigSavingsDay


In [12]:
MIN_NUM_OF_PRODUCTS_TO_SCRAPE = 120

with open(outfile_path,'a') as outfile:
    for k in keyword_list:
        pg_number = 1
        search_rank = 1
        if k == 'EOF':
            break
        
        while True:
            
            if search_rank >= MIN_NUM_OF_PRODUCTS_TO_SCRAPE + 1:
                break
                
            # To account for differnt urls based on page number
            if pg_number == 1:
                url = str("https://www.flipkart.com/search?q="+str(k))
            else:
                url = str("https://www.flipkart.com/search?q="+str(k)+"&page="+ str(pg_number))
            
            data_text = scrape_SearchResult(url)

            if data_text == 'False':
                break
            
            else:
                
                # Extract all product details in a dict 'data' using the extractor file
                data = e.extract(data_text)
                
                if data['url_a'] is not None:
                    urls = data['url_a']
                if data['url_b'] is not None:
                    urls = data['url_b']
                
                # Save html text to file
                html_files_path = str('./ScriptOutput/HTML/'+ str(FileName) + '/' + str(k) +'/Page_'+str(pg_number)+'.html')
                os.makedirs(os.path.dirname(html_files_path), exist_ok=True)
                with open(html_files_path, 'w') as file:
                    file.write(data_text)
                
                for product in urls:
                    print('Product '+ str(search_rank%len(urls)) + ' of '+ str(len(urls)) + ' on this page!')
#                     print(urls)
                    product['SearchResultPosition'] = search_rank
                    search_rank += 1
                    product['SearchKeyword'] = k
                    product['SearchUrl'] = url
                    date = datetime.datetime.now()
                    product['Timestamp'] = date.strftime("%c")
                    product['KeywordType'] = KeywordType
                    if product['Label'] is not None:
                        product['Label'] = 'Flipkart Assured'
                    if 'www.flipkart.' in product['ProductPageUrl']:
                        data = scrape_ProductPage(product['ProductPageUrl'])
                    else:
                        product['ProductPageUrl'] = 'https://www.flipkart.com'+ product['ProductPageUrl']
                        data = scrape_ProductPage(product['ProductPageUrl'])
#                     print(data)
                        
                    if data == 'False':
                        product['Title'] = None
                        product['MRP'] = None
                        product['FlipkartPrice'] = None
                        product['DiscountPercentage'] = None
                        product['Rating'] = None
                        product['RatingCount'] = None
                        product['ProductDescription'] = None
                        product['Breadcrumbs'] = None
                        product['FlipkartAssured'] = None

                        print("Saving Product: %s"%product['Title'])
                        print(product)
                        json.dump(product,outfile)
                        outfile.write("\n")
                        continue

                    product['Title'] = data['Title']
                    product['MRP'] = data['MRP']
                    product['FlipkartPrice'] = data['FlipkartPrice']
                    product['DiscountPercentage'] = data['DiscountPercentage']
                    product['Rating'] = data['Rating']
                    product['ProductDescription'] = data['ProductDescription']
                    product['RatingCount'] = data['RatingCount']
                    if data['FlipkartAssured'] is not None:
                        product['FlipkartAssured'] = 'Flipkart Assured'
                    else:
                        product['FlipkartAssured'] = None

                    product['Breadcrumbs'] = data['Breadcrumbs']

                    product['Seller'] = data['Seller']

                    print("Saving Product: %s"%product['Title'])
                    print(product)
                    json.dump(product,outfile)
                    outfile.write("\n")
                       
                pg_number += 1


Trial no: 1
Product 1 of 40 on this page!
Trial no: 1
Saving Product: D & Y Edge To Edge Tempered Glass for Nintendo Switch (Pack of 1)
{'ProductPageUrl': 'https://www.flipkart.com/d-y-edge-tempered-glass-nintendo-switch/p/itmfdurccmgnkgsb?pid=ACCFDUPJ4XA3ZDZU&lid=LSTACCFDUPJ4XA3ZDZUMRWALA&marketplace=FLIPKART&srno=s_1_1&otracker=search&fm=organic&iid=907b0487-7f60-4f8c-9220-2f7f5deeacc2.ACCFDUPJ4XA3ZDZU.SEARCH&ssid=ob1wjuq1gg0000001597426948629&qH=e8b4e44f4d145914', 'Label': None, 'SearchResultPosition': 1, 'SearchKeyword': 'nintendo+switch', 'SearchUrl': 'https://www.flipkart.com/search?q=nintendo+switch', 'Timestamp': 'Fri Aug 14 23:12:28 2020', 'KeywordType': 'Top 100 Flipkart BigSavingsDay', 'Title': 'D & Y Edge To Edge Tempered Glass for Nintendo Switch (Pack of 1)', 'MRP': '₹ 799', 'FlipkartPrice': '₹269', 'DiscountPercentage': '66% off', 'Rating': '3.6', 'ProductDescription': 'High-quality material The Ultra Clear High Response Screen Protector is made with ultra-transparent (u

Saving Product: Ugreen Back Cover for Nintendo Switch, Nintendo Switch V2 (Black)
{'ProductPageUrl': 'https://www.flipkart.com/ugreen-back-cover-nintendo-switch-switch-v2/p/itmf09c471f62471?pid=ACCFTZ6NGYMFYFHS&lid=LSTACCFTZ6NGYMFYFHS6RJGSW&marketplace=FLIPKART&srno=s_1_6&otracker=search&fm=organic&iid=907b0487-7f60-4f8c-9220-2f7f5deeacc2.ACCFTZ6NGYMFYFHS.SEARCH&ssid=ob1wjuq1gg0000001597426948629&qH=e8b4e44f4d145914', 'Label': None, 'SearchResultPosition': 6, 'SearchKeyword': 'nintendo+switch', 'SearchUrl': 'https://www.flipkart.com/search?q=nintendo+switch', 'Timestamp': 'Fri Aug 14 23:12:31 2020', 'KeywordType': 'Top 100 Flipkart BigSavingsDay', 'Title': 'Ugreen Back Cover for Nintendo Switch, Nintendo Switch V2 (Black)', 'MRP': '₹ 2,999', 'FlipkartPrice': '₹1,199', 'DiscountPercentage': '60% off', 'Rating': None, 'ProductDescription': '??Designed specifically for Nintendo Switch (Not for Lite) ??Made of shock-resistant flexible TPU and scratch-resistant PC materials, protecting your

KeyboardInterrupt: 

In [ ]:
# ProductPageUrl
# Label
# SearchResultPosition
# SearchKeyword
# SearchUrl
# Timestamp
# KeywordType
# Title
# FLipkartPrice
# MRP
# DiscountPercentage
# Rating
# ProductDescription
# RatingCount
# FlipkartAssured
# Breadcrumbs
# Seller